# Plafonnement de la variable à des valeurs maximales et minimales arbitraires

Comme pour la winsorisation, on peut remplacer les valeurs extrêmes par des valeurs plus proches des autres valeurs de la variable, en déterminant les bornes maximum et minimum avec la moyenne plus ou moins l'écart type,ou la règle de proximité interquartile.

Nous pouvons plafonner les deux extrêmes de la distribution ou une seule des queues, selon l'endroit où nous trouvons les valeurs aberrantes dans la variable. Dans cette recette, nous remplacerons les valeurs extrêmes par la moyenne et l'écart-type ou la règle de proximité inter-quartile, en utilisant pandas, NumPy et Feature-engine, et en utilisant l'ensemble de données Boston House Prices de scikit-learn.

Commençons par importer les bibliothèques Python nécessaires :

In [8]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# boston house dataset for the demo
from sklearn.datasets import load_boston

Chargeons l'ensemble de données sur les prix des maisons de Boston à partir de scikit-learn :

In [6]:
# load the the Boston House price data

# load the boston dataset from sklearn
boston_dataset = load_boston()

# create a dataframe with the independent variables
# I will use only 3 of the total variables for this demo

boston = pd.DataFrame(boston_dataset.data,
                      columns=boston_dataset.feature_names)[[
                          'RM', 'LSTAT', 'CRIM'
                      ]]

boston.head()

,RM,LSTAT,CRIM
0,6.575,4.98,0.00632
1,6.421,9.14,0.02731
2,7.185,4.03,0.02729
3,6.998,2.94,0.03237
4,7.147,5.33,0.06905


Créons une fonction pour trouver les limites en utilisant la règle de proximité inter-quartile

In [1]:
def find_skewed_boundaries(df, variable, distance):

    # Let's calculate the boundaries outside which sit the outliers
    # for skewed distributions

    # distance passed as an argument, gives us the option to
    # estimate 1.5 times or 3 times the IQR to calculate
    # the boundaries.

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary

si, à la place, nous voulions trouver les frontières avec la moyenne et l'écart-type, nous pouvons réécrire notre fonction comme suit.

In [4]:
def find_normal_boundaries(df, variable, distance):

    # calculate the boundaries outside which sit the outliers
    # for a Gaussian distribution

    upper_boundary = df[variable].mean() + distance * df[variable].std()
    lower_boundary = df[variable].mean() - distance * df[variable].std()

    return upper_boundary, lower_boundary

Une fois que nous avons créé les fonctions, nous pouvons aller de l'avant et trouver les limites en utilisant soit la moyenne et l'écart type ou l'intervalle interquartile. Dans cette recette, je vais continuer en trouvant les limites en utilisant la moyenne et l'écart type

Si la variable n'est pas distribuée normalement, il peut être plus utile d'utiliser la règle de proximité de l'intervalle interquartile pour trouver les valeurs aberrantes

In [7]:
# find limits for RM

RM_upper_limit, RM_lower_limit = find_normal_boundaries(boston, 'RM', 3)
RM_upper_limit, RM_lower_limit

(8.392485817597757, 4.176782957105816)

In [8]:
# limits for LSTAT

LSTAT_upper_limit, LSTAT_lower_limit = find_normal_boundaries(boston, 'LSTAT', 3)
LSTAT_upper_limit, LSTAT_lower_limit

(34.07624777515244, -8.77012129293899)

In [9]:
# limits for CRIM

CRIM_upper_limit, CRIM_lower_limit = find_normal_boundaries(boston, 'CRIM', 3)
CRIM_upper_limit, CRIM_lower_limit

(29.418158873309714, -22.19111175868521)

In [ ]:
Remplacer les valeurs extrêmes par les limites dans RM

In [10]:
# Now let's replace the outliers by the maximum and minimum limit

boston['RM']= np.where(boston['RM'] > RM_upper_limit, RM_upper_limit,
                       np.where(boston['RM'] < RM_lower_limit, RM_lower_limit, boston['RM']))

In [11]:
# Now let's replace the outliers by the maximum and minimum limit

boston['LSTAT']= np.where(boston['LSTAT'] > LSTAT_upper_limit, LSTAT_upper_limit,
                       np.where(boston['LSTAT'] < LSTAT_lower_limit, LSTAT_lower_limit, boston['LSTAT']))

In [12]:
# Now let's replace the outliers by the maximum and minimum limit

boston['CRIM']= np.where(boston['CRIM'] > CRIM_upper_limit, CRIM_upper_limit,
                       np.where(boston['CRIM'] < CRIM_lower_limit, CRIM_lower_limit, boston['CRIM']))


Nous pouvons limiter de nombreuses variables à la fois, en utilisant le package open source, Feature-engine. Pour ce faire, nous devons charger les bibliothèques et les données comme nous l'avons fait à l'étape 1 dans Ensuite, nous devons importer Feature-engin

In [13]:
# load the the Boston House price data

# load the boston dataset from sklearn
boston_dataset = load_boston()

# create a dataframe with the independent variables
# I will use only 3 of the total variables for this demo

boston = pd.DataFrame(boston_dataset.data,
                      columns=boston_dataset.feature_names)[[
                          'RM', 'LSTAT', 'CRIM'
                      ]]

# add the target
boston['MEDV'] = boston_dataset.target

boston.head()

,RM,LSTAT,CRIM,MEDV
0,6.575,4.98,0.00632,24.0
1,6.421,9.14,0.02731,21.6
2,7.185,4.03,0.02729,34.7
3,6.998,2.94,0.03237,33.4
4,7.147,5.33,0.06905,36.2


In [2]:
! pip install feature_engine

     |████████████████████████████████| 179 kB 5.4 MB/s 
     |████████████████████████████████| 9.5 MB 31.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [5]:
from feature_engine.outliers import Winsorizer

In [9]:
# load the the Boston House price data

# load the boston dataset from sklearn
boston_dataset = load_boston()

# create a dataframe with the independent variables
# I will use only 3 of the total variables for this demo

boston = pd.DataFrame(boston_dataset.data,
                      columns=boston_dataset.feature_names)[[
                          'RM', 'LSTAT', 'CRIM'
                      ]]

# add the target
boston['MEDV'] = boston_dataset.target

boston.head()

,RM,LSTAT,CRIM,MEDV
0,6.575,4.98,0.00632,24.0
1,6.421,9.14,0.02731,21.6
2,7.185,4.03,0.02729,34.7
3,6.998,2.94,0.03237,33.4
4,7.147,5.33,0.06905,36.2


In [10]:
# create the capper

windsorizer = Winsorizer(capping_method ='gaussian', # choose iqr for skewed or gaussian
                          tail='both', # cap left, right or both tails 
                          fold=3,
                          variables=['RM', 'LSTAT', 'CRIM'])

windsorizer.fit(boston)

Winsorizer(capping_method='gaussian', fold=3, missing_values='raise',
           tail='both', variables=['RM', 'LSTAT', 'CRIM'])

In [11]:
boston_t = windsorizer.transform(boston)

In [12]:
# we can inspect the minimum caps for each variable
windsorizer.left_tail_caps_

{'CRIM': -22.19111175868521,
 'LSTAT': -8.77012129293899,
 'RM': 4.176782957105816}

In [13]:

# we can inspect the maximum caps for each variable
windsorizer.right_tail_caps_

{'CRIM': 29.418158873309714,
 'LSTAT': 34.07624777515244,
 'RM': 8.392485817597757}

Dans cette recette, nous avons remplacé les valeurs aberrantes de trois variables dans l'ensemble de données Boston House Prices de scikit-learn. Pour remplacer les valeurs aberrantes, nous avons d'abord identifié ces valeurs en utilisant la moyenne et l'écart type, puis nous avons remplacé les valeurs au-delà de ces limites par les valeurs aux limites.

Nous avons d'abord chargé les données,Pour identifier les valeurs aberrantes dans notre base de données, nous avons créé une fonction pour trouver des limites à l'aide de la règle de proximité de l'intervalle interquartile ou de la moyenne et de l'écart type, respectivement. La fonction prend le data frame et la variable comme arguments et calcule l'intervalle interquartile et les limites max min . 
